In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import BeamDynamics as bd
import SimulationData as sd
import RFTrackTools as rfttools
import copy
import json

In [ ]:
from importlib import reload
reload(bd)
reload(rfttools)

In [ ]:
# %matplotlib inline
# %matplotlib notebook
%matplotlib widget
plt.rcParams['figure.figsize'] = [9.6, 6.4]
defaultColorCycle = plt.rcParams["axes.prop_cycle"].by_key()['color']
# plotFont = {
#     'family' : 'sans-serif',
#     'weight' : 'normal',
#     'size'   : 12
# }
# matplotlib.rc('font', **plotFont)
# plt.rc('legend', fontsize=10)

# Emittance vs. Number of Particles

## Distributions at the End of Capture Linac V0 (After 5 RF Structures, ~ 200 MeV)

### 3 m L-Band Large R (30 mm), Realistic Solenoids Type 1

In [ ]:
# DISTR_REL_PATH_1 = '../../Data/RFTrack/CaptureLinac/CaptureLinacUpTo200MeV_LBandLargeR_RealSolenoids_Type1and2_TargetAt35mm/DistrOut_After1stTracking_6d.sdf_txt'
DISTR_REL_PATH_1 = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2_TuningSolenoid_Ecut40MeV/DistrOut_After2ndTracking_6d.sdf_txt'
# DISTR_REL_PATH_1 = '../../Results_CaptureLinac/LatestSim/DistrOut_After1stTracking_6d.sdf_txt'
FILTER_SPECS_MAIN_BUNCH_1 = 'MainBunch'
beam1 = bd.load_standard_fwf(DISTR_REL_PATH_1)
beam1.describe()

## Plot Full Distributions

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT']
plotDefs = bd.set_plot_defs_from_distrs([beam1], setNames=plotSets)
_ = bd.plot_distr(
    [beam1],
    plotDefs, #markerStyle=['o', 'x'], legendLabels=['Volume + Lattice', 'Volume']
)

## Select Desired Bucket

In [ ]:
mainBunch1 = bd.use_filter_specs_selector(beam1, DISTR_REL_PATH_1, FILTER_SPECS_MAIN_BUNCH_1)
plotDefs = bd.set_plot_defs_from_distrs([mainBunch1], setNames=plotSets)
_ = bd.plot_distr(
    [mainBunch1],
    plotDefs, #markerStyle=['o', 'x'], legendLabels=['Volume + Lattice', 'Volume']
)

In [ ]:
filterSpecs = {}
emitn = {}
emitGeom = {}
emitTraceSpace = {}
alphaTwiss = {}
betaTwiss = {}
gammaTwiss = {}
for planeName in ('x', 'y'):
    emitn[planeName] = bd.compute_emittance(
        mainBunch1, planeName, norm='normalized', filterSpecs=filterSpecs
    )
    emitGeom[planeName] = bd.compute_emittance(
        mainBunch1, planeName, norm='geometric', filterSpecs=filterSpecs
    )
    emitTraceSpace[planeName] = bd.compute_emittance(
        mainBunch1, planeName, norm='tracespace', filterSpecs=filterSpecs
    )
    alphaTwiss[planeName], betaTwiss[planeName], gammaTwiss[planeName] = bd.compute_twiss(
        mainBunch1, planeName, filterSpecs=filterSpecs
    )
    print(
        'emitn_{0:s} = {1:.1f} pi mm mrad, emitGeom_{0:s} = {2:.1f} pi mm mrad, emitTraceSpace_{0:s} = {3:.1f} pi mm mrad.'.format(
            planeName, emitn[planeName], emitGeom[planeName], emitTraceSpace[planeName]
        )
    )
    print(
        'alphaTwiss_{0:s} = {1:.3f}, betaTwiss_{0:s} = {2:.3f} mm, gammaTwiss_{0:s} = {3:.3f} 1/mm.'.format(
            planeName, alphaTwiss[planeName], betaTwiss[planeName], gammaTwiss[planeName]
        )
    )

In [ ]:
# emitRef = emitTraceSpace['x']
# or
refParticle = bd.get_json_entry(DISTR_REL_PATH_1, FILTER_SPECS_MAIN_BUNCH_1, 'RefParticle1')
betaGammaRef = bd.p_to_betagamma(np.array(refParticle['pz']), -11)
print('betaGammaRef = {:.1f}.'.format(betaGammaRef))
emitRef = emitn['x'] / betaGammaRef

In [ ]:
selFa = 4.
gaussianPortions = {1: 0.6827, 2: 0.9545, 3: 0.9973, 4: 0.999937, 6: 0.99999998}
ellipseSpecs = {
    'x': {'alphaTwiss': alphaTwiss['x'], 'betaTwiss': betaTwiss['x']},
    'y': {'alphaTwiss': alphaTwiss['y'], 'betaTwiss': betaTwiss['y']}
}
for planes in (['x'], ['x', 'y']):
    print('\nFiltering on {:d} planes:'.format(len(planes)))
    for Fa in (1., 2., 3., 4., 6.):
        selEllipseSpecs = {k: v for k, v in ellipseSpecs.items() if k in planes}
        distrWithinFaSigma, portionWithinFaSigma = bd.distr_within_ellipse(
            mainBunch1, Fa**2.*emitRef, selEllipseSpecs
        )
        print(
            'Portion within {:.1f} sigma: {:.3f} (vs. {:.3f} for Gaussian).'.format(
                Fa, portionWithinFaSigma, gaussianPortions[Fa]
            )
        )

In [ ]:
refEllipseSpecs = {
    'x': {'alphaTwiss': alphaTwiss['x'], 'betaTwiss': betaTwiss['x']},
    'y': {'alphaTwiss': alphaTwiss['y'], 'betaTwiss': betaTwiss['y']}
}
FaList = np.arange(10., 0, -0.25)  # (4., 3., 2., 1.)
distrWithinRef = []
bunchPortions = []
emitnPortions = []
for Fa in FaList:
    distrWithinRef.append(bd.distr_within_ellipse(
        mainBunch1, Fa**2.*emitRef, refEllipseSpecs
    )[0])
    bunchPortions.append(distrWithinRef[-1].shape[0]/mainBunch1.shape[0])
    emitnPortions.append(bd.compute_emittance(distrWithinRef[-1], 'x'))

In [ ]:
fig, ax = plt.subplots(figsize=(8., 4.5))
ax.plot(bunchPortions, emitnPortions, 'o-')
for ind, txt in enumerate(FaList):
    ax.annotate(txt, (bunchPortions[ind], emitnPortions[ind]))
ax.set_xlim([0, 1])
ax.set_ylim([0, np.max(emitnPortions)])
ax.set_xlabel('Bunch portion')
ax.set_ylabel('Norm. emittance [pimmmrad]')
ax.grid()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4.5))
ax.plot(FaList, bunchPortions, 'o-')
ax.set_xlim([0, np.max(FaList)])
ax.set_ylim([0, np.max(bunchPortions)])
ax.set_xlabel('Fa of cut with Fa^2*rmsEmit ellipse')
ax.set_ylabel('Bunch portion')
ax.grid()

In [ ]:
stepsToPlot = np.arange(6*4, len(distrWithinRef), 4)
FaListToPlot = [FaList[stepInd] for stepInd in stepsToPlot]
plotDefs[1]['lims2'] = [-3., 3.]
plotDefs[1]['displayTable'] = False
ax = bd.plot_distr(
    [mainBunch1, *[distrWithinRef[stepInd] for stepInd in stepsToPlot]], plotDefs,
    legendLabels=[
        'Full bunch',
        *['Within {:.1f} sigma'.format(Fa) for Fa in FaListToPlot]
    ]
)
for ind, Fa in enumerate(FaListToPlot):
    bd.plot_ellipse(
        ax[1][0,0], Fa**2.*emitRef, semiAxisOrder=2,
        alphaTwiss=alphaTwiss['x'], betaTwiss=betaTwiss['x'],
        color='k'  # ax[1][0,0].get_children()[ind+1].get_edgecolor()
    )
    bd.plot_ellipse(
        ax[2][0,0], Fa**2.*emitRef, semiAxisOrder=2,
        alphaTwiss=alphaTwiss['y'], betaTwiss=betaTwiss['y'],
        color='k'  # ax[2][0,0].get_children()[ind+1].get_edgecolor()
    )

<div class="alert alert-block alert-success">
Some good news.
</div>

<div class="alert alert-block alert-warning">
Some warning.
</div>

<div class="alert alert-block alert-danger">
Some danger.
</div>